In [19]:
from sklearn.datasets import load_files

review_train = load_files("aclImdb/train/")
review_test = load_files("aclImdb/test/")

In [30]:
text_train, y_train = review_train.data, review_train.target

In [21]:
text_train[0]

b'Full of (then) unknown actors TSF is a great big cuddly romp of a film.<br /><br />The idea of a bunch of bored teenagers ripping off the local sink factory is odd enough, but add in the black humour that Forsyth & Co are so good at and your in for a real treat.<br /><br />The comatose van driver by itself worth seeing, and the canal side chase is just too real to be anything but funny.<br /><br />And for anyone who lived in Glasgow it\'s a great "Oh I know where that is" film.'

In [22]:
len(text_train)

25000

In [23]:
text_test, y_test = review_test.data, review_test.target

In [24]:
len(text_test)

25000

In [25]:
text_test[0]

b"Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!"

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=5, ngram_range=(2, 2))
X_train = vect.fit(text_train).transform(text_train)
X_test = vect.fit(text_test).transform(text_test)

print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("X_train {}".format(repr(X_train)))
print("X_test {}".format(repr(X_test)))

feature_names = vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))

Vocabulary size: 126529
X_train <75000x332885 sparse matrix of type '<class 'numpy.int64'>'
	with 12160550 stored elements in Compressed Sparse Row format>
X_test <25000x126529 sparse matrix of type '<class 'numpy.int64'>'
	with 3520049 stored elements in Compressed Sparse Row format>
Number of features: 126529


In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(verbose=True), param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)

/home/senpai/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/senpai/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

/home/senpai/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Best cross-validation score: 0.68
Best parameters:  {'C': 0.1}
Best estimator:  LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=True, warm_start=False)


In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

In [39]:
import nltk
nltk.download('punkt')
movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)

moview_counts = movie_vec.fit_transform(text_train)

[nltk_data] Downloading package punkt to /home/senpai/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [40]:
moview_counts

<75000x94342 sparse matrix of type '<class 'numpy.int64'>'
	with 11035609 stored elements in Compressed Sparse Row format>

In [42]:
movie_tfidf = tfidf_transformer.fit_transform(moview_counts)

In [43]:
movie_tfidf.shape

(75000, 94342)

In [53]:
from sklearn.svm import SVC

In [54]:
from sklearn.model_selection import train_test_split
docs_train, docs_test, Y_train, Y_test = train_test_split(
    movie_tfidf, y_train, test_size = 0.20, random_state = 12)

In [55]:
clf = SVC().fit(docs_train, Y_train)

/home/senpai/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [56]:
import sklearn
y_pred = clf.predict(docs_test)
sklearn.metrics.accuracy_score(Y_test, y_pred)


0.6583333333333333

In [52]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
cm

array([[   0,    0, 2576],
       [   0,    1, 2548],
       [   0,    0, 9875]])